In [54]:
import os

import cv2
import numpy as np
import tensorflow as tf
from huggingface_hub import from_pretrained_keras

IMG_SIZE = 224

In [ ]:
backbone = from_pretrained_keras("keras-io/monocular-depth-estimation")

In [55]:
import onnxruntime as ort
session = ort.InferenceSession("../../ckpt/aenet_complete_output.onnx")
session.set_providers(['CUDAExecutionProvider'])
input_name = session.get_inputs()[0].name

/home/pessoal/miniconda3/envs/liveness/lib/python3.12/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [56]:
base_path = "/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/"
live_paths = [
    os.path.join(base_path, "4930/live/504311.png"),
    os.path.join(base_path, "4930/live/511717.png"),
    os.path.join(base_path, "4931/live/494918.png"),
    os.path.join(base_path, "4943/live/494437.png"),
    os.path.join(base_path, "4973/live/494955.png"),
]
spoof_paths = [
    os.path.join(base_path, "4930/spoof/495600.png"),
    os.path.join(base_path, "3613/spoof/541354.png"),
    os.path.join(base_path, "4931/spoof/497474.png"),
    os.path.join(base_path, "4943/spoof/494824.png"),
    os.path.join(base_path, "4973/spoof/502204.png"),
]
bbox_lives = ["".join(path.split(".")[:-1]) + "_BB.txt" for path in live_paths]
bbox_spoofs = ["".join(path.split(".")[:-1]) + "_BB.txt" for path in spoof_paths]

print(live_paths)
print(spoof_paths)
print(bbox_lives)
print(bbox_spoofs)

['/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/live/504311.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/live/511717.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4931/live/494918.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4943/live/494437.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4973/live/494955.png']
['/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4930/spoof/495600.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/3613/spoof/541354.png', '/home/pessoal/Documents/repos/liveness_system/data/celebA-spoof/CelebA_Spoof_/CelebA_Spoof/Data/test/4931/spoof/497474.p

In [57]:
live_images = []
spoof_images = []
bb_lives = []
bb_spoofs = []

for live_path, spoof_path, bb_live, bb_spoof in zip(live_paths, spoof_paths, bbox_lives, bbox_spoofs):
    # Load the images
    live_images.append(cv2.imread(live_path))
    spoof_images.append(cv2.imread(spoof_path))

    # Extract bounding boxes
    with open(bb_live, "r") as file:
        bb_lives.append(list(map(int, file.readlines()[0].split()[:-1])))

    with open(bb_spoof, "r") as file:
        bb_spoofs.append(list(map(int, file.readlines()[0].split()[:-1])))

print(live_images[0].shape)
print(spoof_images[0].shape)
print(bb_lives[0])
print(bb_spoofs[0])
bbox_lives = bb_lives
bbox_spoofs = bb_spoofs

(495, 500, 3)
(450, 600, 3)
[54, 57, 96, 129]
[33, 57, 94, 115]


In [58]:
for i in range(len(live_images)):
    # Resize images
    live_images[i] = cv2.resize(live_images[i], (IMG_SIZE, IMG_SIZE))
    spoof_images[i] = cv2.resize(spoof_images[i], (IMG_SIZE, IMG_SIZE))

    # Crop faces and show cropped images
    bbox_live = bbox_lives[i]
    bbox_spoof = bbox_spoofs[i]

    live_images[i] = live_images[i][
        bbox_live[1] : bbox_live[1] + bbox_live[3],
        bbox_live[0] : bbox_live[0] + bbox_live[2],
    ]
    spoof_images[i] = spoof_images[i][
        bbox_spoof[1] : bbox_spoof[1] + bbox_spoof[3],
        bbox_spoof[0] : bbox_spoof[0] + bbox_spoof[2],
    ]

In [59]:
for i in range(len(live_images)):
    # Preprocess the cropped images
    live_images[i] = cv2.resize(live_images[i], (IMG_SIZE, IMG_SIZE))
    spoof_images[i] = cv2.resize(spoof_images[i], (IMG_SIZE, IMG_SIZE))
    cv2.imwrite("Live_Image.png", live_images[i])
    cv2.imwrite("Spoof_Image.png", spoof_images[i])
    live_images[i] = cv2.cvtColor(live_images[i], cv2.COLOR_BGR2RGB)
    spoof_images[i] = cv2.cvtColor(spoof_images[i], cv2.COLOR_BGR2RGB)

    live_images[i] = live_images[i].astype(np.float32)
    spoof_images[i] = spoof_images[i].astype(np.float32)


    #--------------------------------------------------------------------------
    # Old normalization
    # # z-score normalization + scaling to [-1, 1]

    # live_image = (live_image - np.mean(live_image)) / np.std(live_image)
    # spoof_image = (spoof_image - np.mean(spoof_image)) / np.std(spoof_image)

    # live_image = (
    #     (live_image - np.min(live_image))
    #     / (np.max(live_image) - np.min(live_image))
    # )
    # spoof_image = (
    #     (spoof_image - np.min(spoof_image))
    #     / (np.max(spoof_image) - np.min(spoof_image))
    # )

    # live_image = live_image * 2.0 - 1.0
    # spoof_image = spoof_image * 2.0 - 1.0

    # print(np.max(live_image))
    # print(np.min(live_image))
    # print(np.max(spoof_image))
    # print(np.min(spoof_image))
    #--------------------------------------------------------------------------


    # # Normalize for tensorflow model
    # live_images[i] /= 127.5
    # live_images[i] -= 1.0
    # spoof_images[i] /= 127.5
    # spoof_images[i] -= 1.0

    # # Make predictions
    # live_images[i] = tf.expand_dims(live_images[i], axis=0)
    # spoof_images[i] = tf.expand_dims(spoof_images[i], axis=0)
    # live_depth_map = backbone.predict(live_images[i])[0]
    # spoof_depth_map = backbone.predict(spoof_images[i])[0]


    # Normalize for torch model and make predictions
    live_images[i] /= 255.
    spoof_images[i] /= 255.
    live_images[i] = np.transpose(live_images[i], (2, 0, 1))
    spoof_images[i] = np.transpose(spoof_images[i], (2, 0, 1))
    live_images[i] = np.expand_dims(live_images[i], axis=0)
    spoof_images[i] = np.expand_dims(spoof_images[i], axis=0)
    result_live = session.run(None, {input_name: live_images[i].astype(np.float32)})
    result_spoof = session.run(None, {input_name: spoof_images[i].astype(np.float32)})

    x_live, live_depth_map, reflect_map_live, x_live_attribute_live, x_attack_live, x_light_live = result_live
    x_spoof, spoof_depth_map, reflect_map_spoof, x_spoof_attribute_spoof, x_attack_spoof, x_light_spoof = result_spoof

    print(x_live.shape)
    print(live_depth_map.shape)
    print(reflect_map_live.shape)
    print(x_live_attribute_live.shape)
    print(x_attack_live.shape)
    print(x_light_live.shape)

    print(x_live)
    print(x_spoof)

    live_depth_map = np.transpose(live_depth_map[0], (1, 2, 0))
    spoof_depth_map = np.transpose(spoof_depth_map[0], (1, 2, 0))


    # Normalize the depth maps
    live_depth_map = (
        (live_depth_map - np.min(live_depth_map))
        / (np.max(live_depth_map) - np.min(live_depth_map))
        * 255.0
    )
    spoof_depth_map = (
        (spoof_depth_map - np.min(spoof_depth_map))
        / (np.max(spoof_depth_map) - np.min(spoof_depth_map))
        * 255.0
    )

    # Convert the masks to uint8
    live_depth_map = live_depth_map.astype(np.uint8)
    spoof_depth_map = spoof_depth_map.astype(np.uint8)

    cv2.imwrite("Live_Depth_Map.png", live_depth_map)
    cv2.imwrite("Spoof_Depth_Map.png", spoof_depth_map)

    # Show the predicted depth maps
    cv2.imshow("Live Depth Map", live_depth_map)
    cv2.imshow("Spoof Depth Map", spoof_depth_map)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

(1, 2)
(1, 1, 14, 14)
(1, 3, 14, 14)
(1, 40)
(1, 11)
(1, 5)
[[ 1.3083305 -1.2484285]]
[[-5.7333527  5.991271 ]]
(1, 2)
(1, 1, 14, 14)
(1, 3, 14, 14)
(1, 40)
(1, 11)
(1, 5)
[[ 2.8779607 -2.9063878]]
[[-6.8667774  6.9544535]]
(1, 2)
(1, 1, 14, 14)
(1, 3, 14, 14)
(1, 40)
(1, 11)
(1, 5)
[[-3.843918   3.7828765]]
[[-9.21073   9.287416]]


/tmp/ipykernel_377119/983765357.py:87: RuntimeWarning: invalid value encountered in divide
  (spoof_depth_map - np.min(spoof_depth_map))
/tmp/ipykernel_377119/983765357.py:94: RuntimeWarning: invalid value encountered in cast
  spoof_depth_map = spoof_depth_map.astype(np.uint8)


(1, 2)
(1, 1, 14, 14)
(1, 3, 14, 14)
(1, 40)
(1, 11)
(1, 5)
[[-0.6511588   0.57541764]]
[[-7.6500654  7.7122574]]
(1, 2)
(1, 1, 14, 14)
(1, 3, 14, 14)
(1, 40)
(1, 11)
(1, 5)
[[ 4.3500648 -4.3044214]]
[[-7.0702085  7.229003 ]]
